# Exercise: Flood Analysis in Opole (2024) using Sentinel-1 SAR Data

In this exercise, you will compare radar backscatter from Sentinel-1 before and after the 2024 flood in Opole, Poland.

**Objective:**
- Load Sentinel-1 VV data before and after the flood
- Convert radar intensity to decibels (dB)
- Calculate the difference between the two dates
- Identify areas with a significant decrease in VV (possible flood zones)

## Step 1: Import Earth Engine and geemap

In [2]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
opole = ee.Geometry.Point([17.931, 50.675])


Successfully saved authorization token.


## Step 2: Load Sentinel-1 VV images (before and after flood)

In [3]:
# TODO: Load Sentinel-1 collection for before and after flood
# Use filterBounds, filterDate, and select 'VV' band
# Apply .mean() to get average image from each period
# Load Sentinel-1 VV images before and after the flood
# Adjust dates as needed for your analysis

# Define date ranges
pre_flood_start = '2024-09-05'
pre_flood_end = '2024-09-10'
post_flood_start = '2024-09-16'
post_flood_end = '2024-09-20'

# Load Sentinel-1 VV images before the flood
s1_pre = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(opole)
    .filterDate(pre_flood_start, pre_flood_end)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .select('VV')
    .mean()
)

# Load Sentinel-1 VV images after the flood
s1_post = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(opole)
    .filterDate(post_flood_start, post_flood_end)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .select('VV')
    .mean()
)

s1_pre

In [ ]:
# Visualization parameters (adjust as needed)
vis_params = {
    'min': -25,
    'max': 0,
    'palette': ['black', 'gray', 'white']
}

# Create and display interactive map
Map = geemap.Map(center=[50.675, 17.931], zoom=11)

Map.addLayer(s1_pre, vis_params, 'Sentinel-1 Pre-Flood VV (dB)')
Map.addLayer(s1_post, vis_params, 'Sentinel-1 Post-Flood VV (dB)')
Map.addLayer(opole, {}, 'Opole AOI')
Map.addLayerControl()
Map


Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Step 3: Convert to dB scale

In [5]:
# TODO: Use .log10().multiply(10) to convert before/after images to dB
s1_pre_db = s1_pre.log10().multiply(10)
s1_post_db = s1_post.log10().multiply(10)

## Step 4: Calculate difference and threshold potential flood areas

In [11]:
# TODO: Subtract after_dB from before_dB
diff_db = s1_post_db.subtract(s1_pre_db)

# Threshold to mask potential flood areas (tune threshold)
flood_mask = diff_db.gt(0.0001)  # Change threshold as needed

Map = geemap.Map(center=[50.675, 17.931], zoom=11)

Map.addLayer(s1_pre, vis_params, 'Sentinel-1 Pre-Flood VV (dB)')
Map.addLayer(s1_post, vis_params, 'Sentinel-1 Post-Flood VV (dB)')
Map.addLayer(diff_db, vis_params, 'diff')
Map.addLayer(opole, {}, 'Opole AOI')
Map.addLayerControl()
Map

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Step 5: Visualize the results

In [7]:
# Visualization parameters
vis_params = {
    'min': -5,
    'max': 5,
    'palette': ['blue', 'yellow', 'red']
}

# Create interactive map
Map = geemap.Map(center=[50.675, 17.931], zoom=11)
#Map.addLayer(s1_pre, vis_params, 'Sentinel-1 Pre-Flood VV (dB)')
#Map.addLayer(s1_post, vis_params, 'Sentinel-1 Post-Flood VV (dB)')
Map.addLayer(diff_db, vis_params,'Difference (dB)')
Map.addLayer(diff_db, vis_params, 'Backscatter Difference (dB)')
Map.addLayer(flood_mask.selfMask(), {'palette': ['blue']}, 'Flood Mask')
Map.addLayer(opole, {}, 'Opole AOI')
Map.addLayerControl()
Map

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…